# Inference notebook

In [1]:
import sys


def in_kaggle():
    return "kaggle_web_client" in sys.modules

In [2]:
import logging


def init_logger():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    handler1 = logging.StreamHandler(stream=sys.stdout)
    handler1.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] [%(module)s] %(message)s"))
    # handler2 = logging.FileHandler(filename="train.log")
    # handler2.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] [%(module)s] %(message)s"))
    logger.addHandler(handler1)
    # logger.addHandler(handler2)
    return logger


log = init_logger()

In [3]:
##### INSERT SOURCE CODE HERE FOR SUBMISSION #####

if in_kaggle():
    sys.path.append("../input/nptyping")
    sys.path.append("../input/typish")
else:
    sys.path.append("..")
    sys.path.append("../../inputs")

In [4]:
if in_kaggle():
    !pip install -qq --no-index --find-links ../input/faissgpu17 -r ../input/faissgpu17/requirements.txt

    !pip install -qq ../input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl
    
    # https://www.kaggle.com/speeddemon/install-hydra-offline-from-dataset
    !cp -r /kaggle/input/hydracore105 /kaggle/working
    !mv /kaggle/working/hydracore105/antlr4-python3-runtime-4.8.tar.gz.tmp /kaggle/working/hydracore105/antlr4-python3-runtime-4.8.tar.gz
    !ls /kaggle/working/hydracore105

    !pip install -qq /kaggle/working/hydracore105/* --ignore-installed PyYAML
    
    sys.path.append("../input/omegaconf/omegaconf-master")
    omega_conf_path = "config/main.yaml"
    
else:
    omega_conf_path = "../config/main.yaml"

In [5]:
from omegaconf import OmegaConf

c = OmegaConf.load(omega_conf_path)

c.settings.debug = False
c.wandb.enabled = False

if in_kaggle():
    c.settings.gpus = "0"

    c.settings.dirs.working = "."
    c.settings.dirs.input = "../input/ubiquant-market-prediction/"
    c.settings.dirs.input_minimal = "../input/ump-train-minimal/"
    c.settings.dirs.feature = "../input/ubiquant-parquet/"
    c.settings.dirs.preprocess = "../input/ump-preprocess/"

    pretraind_dir = "../input/ump-models"

else:
    c.settings.dirs.working = ".."
    c.settings.dirs.input = "../../inputs/"
    c.settings.dirs.input_minimal = "../../datasets/inputs/"
    c.settings.dirs.preprocess = "../../inputs/preprocess/"

    pretraind_dir = "../../datasets/trainings"

In [6]:
pretrained = f"""

- dir: {pretraind_dir}/2022-03-11_20-25-09/fold1/
  model: ump_1
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-11_20-25-15/fold2/
  model: ump_1
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-11_20-26-13/fold4/
  model: ump_1
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-11_20-26-19/fold5/
  model: ump_1
  model_input: 300
  n_class: 1

- dir: {pretraind_dir}/2022-03-12_21-58-26/fold1/
  model: ump_1dcnn
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-12_21-58-32/fold2/
  model: ump_1dcnn
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-12_21-59-36/fold4/
  model: ump_1dcnn
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-12_21-59-41/fold5/
  model: ump_1dcnn
  model_input: 300
  n_class: 1

"""

pretrained_2 = f"""

- dir: {pretraind_dir}/2022-03-17_20-47-04/fold1/
  model: ump_lstm
  model_input: 300
  n_class: 10
  feature_set: ["f001", "f902"]
- dir: {pretraind_dir}/2022-03-17_20-47-09/fold2/
  model: ump_lstm
  model_input: 300
  n_class: 10
  feature_set: ["f001", "f902"]
- dir: {pretraind_dir}/2022-03-17_20-47-41/fold4/
  model: ump_lstm
  model_input: 300
  n_class: 10
  feature_set: ["f001", "f902"]
- dir: {pretraind_dir}/2022-03-17_20-47-47/fold5/
  model: ump_lstm
  model_input: 300
  n_class: 10
  feature_set: ["f001", "f902"]
  
"""


pretrained_lgb = f"""

- dir: {pretraind_dir}/2022-03-13_20-45-32/fold1/ 
  model: lightgbm
  n_class: 1
- dir: {pretraind_dir}/2022-03-13_20-45-32/fold2/ 
  model: lightgbm
  n_class: 1
- dir: {pretraind_dir}/2022-03-13_20-45-32/fold4/ 
  model: lightgbm
  n_class: 1
- dir: {pretraind_dir}/2022-03-13_20-45-32/fold5/ 
  model: lightgbm
  n_class: 1

- dir: {pretraind_dir}/2022-03-18_10-49-32/fold1/tabnet/ 
  model: tabnet
  n_class: 1
- dir: {pretraind_dir}/2022-03-18_10-49-32/fold2/tabnet/ 
  model: tabnet
  n_class: 1
- dir: {pretraind_dir}/2022-03-18_10-49-32/fold4/tabnet/ 
  model: tabnet
  n_class: 1
- dir: {pretraind_dir}/2022-03-18_10-49-32/fold5/tabnet/ 
  model: tabnet
  n_class: 1

"""

_pretrained = f"""

- dir: {pretraind_dir}/2022-03-11_20-25-04/fold0/
  model: ump_1
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-11_20-26-08/fold3/
  model: ump_1
  model_input: 300
  n_class: 1

- dir: {pretraind_dir}/2022-03-12_21-58-21/fold0/
  model: ump_1dcnn
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-12_21-59-31/fold3/
  model: ump_1dcnn
  model_input: 300
  n_class: 1

- dir: {pretraind_dir}/2022-03-17_20-46-59/fold0/
  model: ump_lstm
  model_input: 300
  n_class: 10
  feature_set: ["f001", "f902"]
- dir: {pretraind_dir}/2022-03-17_20-47-36/fold3/
  model: ump_lstm
  model_input: 300
  n_class: 10
  feature_set: ["f001", "f902"]

- dir: {pretraind_dir}/2022-03-13_20-45-32/fold0/ 
  model: lightgbm
  n_class: 1
- dir: {pretraind_dir}/2022-03-13_20-45-32/fold3/ 
  model: lightgbm
  n_class: 1

- dir: {pretraind_dir}/2022-03-18_10-49-32/fold0/tabnet/ 
  model: tabnet
  n_class: 1
- dir: {pretraind_dir}/2022-03-18_10-49-32/fold3/tabnet/ 
  model: tabnet
  n_class: 1

"""

c.params.pretrained = OmegaConf.create(pretrained)
c.params.pretrained_2 = OmegaConf.create(pretrained_2)
c.params.pretrained_lgb = OmegaConf.create(pretrained_lgb)

In [7]:
log.info(OmegaConf.to_yaml(c))

2022-03-18 20:01:04,095 [INFO] [3244290467] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: false
  entity: imokuri
  project: ump
  dir: ${hydra:runtime.cwd}/../cache
  group: default
settings:
  print_freq: 100
  gpus: '6'
  dirs:
    working: ..
    input: ../../inputs/
    input_minimal: ../../datasets/inputs/
    feature: ${settings.dirs.input}features/
    preprocess: ../../inputs/preprocess/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
  training_method: nn
params:
  seed: 7440
  n_class: 1
  preprocess: []
  pca_n_components: 50
  n_fold: 5
  skip_training: false
  epoch: 10
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: combinational_purged
  group_name: investment_id
  time_n

In [8]:
import gc
import traceback

import numpy as np
import src.utils as utils
import ubiquant
from src.feature_store import Store
from src.features.base import get_feature
from src.make_feature import make_feature
from src.make_model import load_model
from src.run_loop import inference, inference_lightgbm

In [9]:
utils.fix_seed(c.params.seed)
utils.debug_settings(c)
device = utils.gpu_settings(c)

In [10]:
feature_set = ["f000"]
feature_set = list(sorted(list(set(feature_set))))
log.info(f"feature set: {feature_set}")

feature_set_2 = ["f001"]
feature_set_2 = list(sorted(list(set(feature_set_2))))
log.info(f"feature set 2: {feature_set_2}")

# feature_func = [get_feature(f) for f in feature_set]

2022-03-18 20:01:06,061 [INFO] [573233744] feature set: ['f000']
2022-03-18 20:01:06,062 [INFO] [573233744] feature set 2: ['f001']


In [11]:
if c.params.preprocess:
    store = Store.train(c)
else:
    store = Store.empty()

In [12]:
models = load_model(c, device, c.params.pretrained)
models_2 = load_model(c, device, c.params.pretrained_2)
models_lgb = load_model(c, device, c.params.pretrained_lgb)

Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda


In [13]:
env = ubiquant.make_env()  # initialize the environment
iter_test = env.iter_test()  # an iterator which loops over the test set and sample submission

In [14]:
# feature_cols = [f"f_{n}" for n in range(300)]

In [15]:
%%time

for test_df, sample_prediction_df in iter_test:
    gc.collect()

    # log.info(test_df["investment_id"].dtype)  # int16
    # log.info(test_df[feature_cols].values.dtype)  # float64

    predictions_ = []

    try:
        # assert len(test_df["investment_id"].unique()) == len(test_df["investment_id"]), "investment_id is not unique."

        if c.params.use_feature:
            store.update(test_df.values)

            pred_df = make_feature(
                test_df,
                store,
                feature_set,
                load_from_store=False,
                save_to_store=False,
                debug=c.settings.debug,
            )
            # assert len(test_df) == len(pred_df), "test_df and pred_df do not same size."
            # assert list(pred_df.columns) == feature_cols, "pred_df has feature_cols columns."
            # assert np.array_equal(
            #     test_df[feature_cols].astype("float32").values, pred_df.values
            # ), "Default features do not match between test_df and pred_df."

            if c.params.pretrained:
                c.params.n_class = c.params.pretrained[0].n_class
                preds = inference(c, pred_df, device, models)
                predictions_.append(preds)

            if c.params.pretrained_2:
                pred_df_2 = make_feature(
                    test_df,
                    store,
                    feature_set_2,
                    load_from_store=False,
                    save_to_store=False,
                    debug=c.settings.debug,
                )

                c.params.n_class = c.params.pretrained_2[0].n_class
                preds = inference(c, pred_df_2, device, models_2)
                predictions_.append(preds)

            if c.params.pretrained_lgb:
                preds_lgb = inference_lightgbm(pred_df, models_lgb)
                predictions_.append(preds_lgb)

        else:
            if c.params.pretrained:
                preds_ = inference(c, test_df, device, models)
                predictions_.append(preds_)

            if c.params.pretrained_lgb:
                preds_lgb_ = inference_lightgbm(test_df, models_lgb)
                predictions_.append(preds_lgb_)

        # assert np.array_equal(preds_, preds), "Predictions do not match between test_df and pred_df."
        # assert np.array_equal(preds_lgb_, preds_lgb), "LightGBM predictions do not match between test_df and pred_df."

        predictions = np.hstack(predictions_)
        # predictions = np.hstack([preds_, preds_lgb_])
        log.info(f"predictions size: {predictions.shape}")

        sample_prediction_df["target"] = np.nanmean(predictions, axis=1)

        # sample_prediction_df.fillna({"target": 0}, inplace=True)

        # if c.params.use_feature:
        #     store.update_post(sample_prediction_df[["row_id", "target"]].values)

    except Exception as e:
        log.warning(traceback.format_exc())
        raise

    env.predict(sample_prediction_df)  # register your predictions

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
2022-03-18 20:01:14,211 [INFO] [<timed exec>] predictions size: (2, 20)
2022-03-18 20:01:17,882 [INFO] [<timed exec>] predictions size: (3, 20)
2022-03-18 20:01:21,493 [INFO] [<timed exec>] predictions size: (3, 20)
2022-03-18 20:01:25,115 [INFO] [<timed exec>] predictions size: (1, 20)
CPU times: user 32.3 s, sys: 8.96 s, total: 41.2 s
Wall time: 14.6 s
